In [9]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Date
import pandas as pd

# Create a Dash app
app = dash.Dash(__name__)

# SQL Server connection details
server = '(localdb)\\MSSQLLOCALDB'
database = 'VA24-Staging'
driver = 'ODBC Driver 17 for SQL Server'

# Create a connection string
connection_string = f'mssql+pyodbc://@{server}/{database}?driver={driver}'

# Create SQLAlchemy engine
engine = create_engine(connection_string)

# Define the table structure
metadata = MetaData()

va_test_table = Table('VA-Test24', metadata,
                      Column('Record_ID', Integer),
                      Column('Site', String),
                      Column('Para', String),
                      Column('Structure', String),
                      Column('Block', String),
                      Column('Death_5_Years', String),
                      Column('Gender', String),
                      Column('Name', String),
                      Column('Age', Integer),
                      Column('Death_Start_Date', Date),
                      Column('Death_End_Date', Date))

# Create the table in the database if it doesn't exist
metadata.create_all(engine)

# Define the layout of the app
app.layout = html.Div(
    style={
        'backgroundColor': '#E1306C',  # Dark grey background
        'color': 'black',  # Black font color
        'display': 'flex',
        'alignItems': 'center',
        'justifyContent': 'center',
        'height': '100vh',
        'padding': '20px'
    },
    children=[
        html.Div(
            style={'backgroundColor': ' #405DE6', 'padding': '10px', 'borderRadius': '8px', 'width': '60%'},
            children=[
                html.Label('Record_Id:', style={'color': 'white'}),
                dcc.Input(
                    id='record_id',
                    placeholder='Please Enter Your Record_ID',
                    type='number',  # Correct the type to 'number'
                    value='',
                    style={'width': '100%', 'height': '15px', 'textAlign': 'center', 'marginBottom': '5px'}
                ),
                
                html.Label('Site:', style={'color': 'white'}),
                dcc.Input(
                    id='site',
                    placeholder='Please Enter Your Site Name',
                    type='text',
                    value='',
                    style={'width': '100%', 'height': '15px', 'textAlign': 'center', 'marginBottom': '5px'}
                ),
                
                html.Label('Para:', style={'color': 'white'}),
                dcc.Input(
                    id='para',
                    placeholder='Please Enter Your Para Code',
                    type='text',
                    value='',
                    style={'width': '100%', 'height': '15px', 'textAlign': 'center', 'marginBottom': '5px'}
                ),
                
                html.Label('Structure:', style={'color': 'white'}),
                dcc.Input(
                    id='structure',
                    placeholder='Please enter your structure',
                    type='text',
                    value='',
                    style={'width': '100%', 'height': '15px', 'textAlign': 'center', 'marginBottom': '5px'}
                ),
                
                html.Label('Block:', style={'color': 'white'}),
                dcc.Input(
                    id='block',
                    placeholder='Please enter your block',
                    type='text',
                    value='',
                    style={'width': '100%', 'height': '15px', 'textAlign': 'center', 'marginBottom': '5px'}
                ),
                
                html.Label('Q.1. Did any death of above 5 years occur in your household from 1 Jan 2024 to 31-Dec-2024?', style={'color': 'white'}),
                dcc.RadioItems(
                    id='death-5-years',
                    options=[
                        {'label': 'Yes', 'value': 'Y'},
                        {'label': 'No', 'value': 'N'}
                    ],
                    value=None,
                    style={'marginBottom': '15px'}
                ),
                
                html.Label('Q.2. What was the gender of the deceased?', style={'color': 'white'}),
                dcc.RadioItems(
                    id='gender',
                    options=[
                        {'label': 'Male', 'value': 'M'},
                        {'label': 'Female', 'value': 'F'}
                    ],
                    value=None,
                    style={'marginBottom': '5px'}
                ),
                
                html.Label('Q.3. What was the name of the deceased?', style={'color': 'white'}),
                dcc.Input(
                    id='name',
                    placeholder='Please enter name of Deceased',
                    type='text',
                    value='',
                    style={'width': '100%', 'height': '15px', 'textAlign': 'center', 'marginBottom': '15px'}
                ),
                
                html.Label('Q.4. What was the age of the deceased?', style={'color': 'white'}),
                dcc.Input(
                    id='age',
                    placeholder='Please enter age',
                    type='number',
                    value='',
                    style={'width': '100%', 'height': '15px', 'border': '2px solid red', 'textAlign': 'center', 'marginBottom': '15px'}
                ),
                
                html.Label('Q.5. When Did the Deceased die?', style={'color': 'white'}),
                dcc.DatePickerRange(
                    id='death-date',
                    start_date='2024-01-01',
                    end_date='2024-12-31',
                    display_format='DD-MM-YYYY',
                    style={'width': '100%', 'marginBottom': '15px'}
                ),
                
                html.Button('Submit', id='submit-button', n_clicks=0,  style={
                        'backgroundColor': '#FD1D1D',  # Green background
                        'color': 'white',  # White text
                        'border': 'none',  # No border
                        'borderRadius': '50px',  # Round corners
                        'padding': '10px 20px',  # Padding
                        'fontSize': '16px',  # Font size
                        'cursor': 'pointer',  # Pointer cursor on hover
                        'width': '70%',  # Full width
                        'textAlign': 'center',  # Center text
                        'marginTop': '15px'  # Margin on top
                    }),
                html.Div(id='alert-message', style={'color': 'green', 'fontWeight': 'bold', 'marginTop': '20px'})
            ]
        )
    ]
)

# Callback to handle form submission and insert data into the database
@app.callback(
    Output('alert-message', 'children'),
    Input('submit-button', 'n_clicks'),
    State('record_id', 'value'),
    State('site', 'value'),
    State('para', 'value'),
    State('structure', 'value'),
    State('block', 'value'),
    State('death-5-years', 'value'),
    State('gender', 'value'),
    State('name', 'value'),
    State('age', 'value'),
    State('death-date', 'start_date'),
    State('death-date', 'end_date')
)
def handle_form_submission(n_clicks, record_id, site, para, structure, block, death_5_years, gender, name, age, start_date, end_date):
    if n_clicks > 0:
        try:
            # Insert form data into the database
            with engine.connect() as conn:
                query = """
                INSERT INTO [VA-Test24] (Record_ID, Site, Para, Structure, Block, Death_5_Years, Gender, Name, Age, Death_Start_Date, Death_End_Date)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """
                conn.execute(query, (record_id, site, para, structure, block, death_5_years, gender, name, age, start_date, end_date))
            return "Form submitted successfully!"
        except Exception as e:
            return f"Error: {str(e)}"
    return ""

if __name__ == '__main__':
    app.run_server(debug=True, port=8052)  # Use port 8052 or any other available port
